In [1]:
import json
import time
from dotenv import load_dotenv

load_dotenv()

import sys
import os


from azure.identity import DefaultAzureCredential
from crm_store import CRMStore

from conversation_store import ConversationStore
from genai_vanilla_agents.workflow import Workflow
from genai_vanilla_agents.conversation import Conversation
from agents.fsi_insurance.group_chat import create_group_chat_insurance
from agents.fsi_banking.group_chat import create_group_chat_banking

key = DefaultAzureCredential()
db = CRMStore(
        url=os.getenv("COSMOSDB_ENDPOINT"),
        key=key,
        database_name=os.getenv("COSMOSDB_DATABASE_NAME"),
        container_name=os.getenv("COSMOSDB_CONTAINER_CLIENT_NAME")
    )

import logging
# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.WARN)
logging.getLogger("genai_vanilla_agents.agent").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.team").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.user").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.workflow").setLevel(logging.DEBUG)

logging.getLogger("agents.fsi_banking.group_chat").setLevel(logging.DEBUG)

### Single Agent Selection from Team

In [2]:
team_query_single = 'Provide me the portfolio positions of my client Pete Mitchell in a table format'
team = create_group_chat_banking(team_query_single)

DEBUG:agents.fsi_banking.group_chat:agent team strategy decision = single
DEBUG:genai_vanilla_agents.team:[Team group_chat] initialized with agents: {'Customer': <genai_vanilla_agents.user.User object at 0x7fea9c5c8f10>, 'Planner': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5baec0>, 'CRM': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5caa70>, 'Funds': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5c9ea0>, 'CIO': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5bb460>, 'News': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5c8a30>}


In [ ]:
team_query_single = 'I understand that coverage applies for domestic destinations beyond 40km. If something happens on the way to the destination but within 40km, is it covered?'
team = create_group_chat_insurance(team_query_single)

### Multi Agent Collaboration from PlannedTeam

In [ ]:
team_query_multiple = 'Provide me the portfolio positions of my client Pete Mitchell in a table format and for each of the position search for today relevant market news if any.'
team = create_group_chat_banking(team_query_multiple)


In [ ]:
workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_single)


print(f"Plan = {team.plan}")
workflow.conversation.messages

In [ ]:
team_query_single = 'I was referring to the Ztravel Plus policy'
team = create_group_chat_insurance(team_query_single)

workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_single)


print(f"Plan = {team.plan}")
workflow.conversation.messages

### Testing AM Research agents

In [ ]:

from agents.fsi_am_research.group_chat import create_group_chat_am
import logging
# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.WARN)
logging.getLogger("genai_vanilla_agents.agent").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.team").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.user").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.workflow").setLevel(logging.DEBUG)

logging.getLogger("agents.fsi_am_research.group_chat").setLevel(logging.DEBUG)

input_email = """ 
Dear Alberto,

InvestAgency ABC is pleased to provide an analysis and outlook on Microsoft Corporation (MSFT) following the recent Fiscal Year 2024 Q4 earnings report.

Summary of Microsoft’s FY24 Q4 Performance:

Microsoft closed its fiscal year with strong performance:

Annual Revenue: Over $245 billion, a 15% year-over-year increase.
Cloud Revenue: Reached $135 billion, growing by 23%.
Key Drivers: The company emphasized growth driven by its Azure and AI offerings, which led to increased market share. Microsoft’s investments in global data centers and AI capabilities, including new accelerators from AMD, NVIDIA, and their own Azure Maia, highlight their commitment to long-term growth in this sector.
CEO Satya Nadella and CFO Amy Hood also emphasized their focus on driving innovation across Microsoft's portfolio while managing costs strategically, with an eye on long-term profitability.

Recommendation: Given Microsoft's robust growth in both revenue and strategic investments in AI and cloud infrastructure, we view MSFT as a strong asset to hold with potential for long-term appreciation. Its position in the AI-driven market and continued cloud dominance make it an attractive investment. For new investors, this stock could be a good buy for portfolio diversification.

Best regards,
InvestAgency ABC
"""

team_query_multiple = f""" 
Given the following email content as input {input_email}, create a summarized reccomendation for this stock or asset by performing: 
- a news and articles research, 
- a sentiment analysis on the stock or asset mentioned, 
- include in-house view from our CIO if any that are relevant to the stock or asset.
"""
team = create_group_chat_am(team_query_multiple)

workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_multiple)

print(f"Plan = {team.plan}")
workflow.conversation.messages



### Testing banking Team chat for repetitive agents loops

In [ ]:
history = []
team_query_single = 'Show Pete Mitchell portfolio performance and suggest any rebalancing options based on recent CIO views.'
team = create_group_chat_banking(team_query_single)

workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_single)

workflow.conversation.messages

In [3]:
history = workflow.conversation.messages
history = history + workflow.conversation.messages
#history

In [ ]:
team_query_single = 'What are our CIO (Chief Investement Office) views on AI opportunities?'
team = create_group_chat_banking(team_query_single)

conv_history = Conversation(messages=history, variables={})
workflow = Workflow(askable=team, conversation=conv_history)
workflow.run(team_query_single)

workflow.conversation.messages

In [5]:
history = workflow.conversation.messages
history = history + workflow.conversation.messages
#history

In [ ]:
team_query_single = 'Are there any relevant news today from the market about the positions on Pete portfolio?'
team = create_group_chat_banking(team_query_single)

conv_history = Conversation(messages=history, variables={})
workflow = Workflow(askable=team, conversation=conv_history)
workflow.run(team_query_single)

workflow.conversation.messages